# Welcome to the start of your adventure in Agentic AI

In [ ]:
# %pip install beautifulsoup4

# Example: Using Groq as an OpenAI-compatible provider
import os
import json
import requests
import gradio as gr
from bs4 import BeautifulSoup
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv(override=True)
GROQ_BASE_URL = "https://api.groq.com/openai/v1"
groq_api_key = os.getenv("GROQ_API_KEY")  # Add your Groq API key to your .env file
openai = OpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)
system_prompt = [{"role": "system", "content": """
                  You are a helpful assistant. First ask for the link to continue chat. If they don't provide them ask it again and again.
                  If link is not provided ask it again calmly.
                  Once the link is provided and tool it called, answers further question from the content."""},]
messages = system_prompt
response = openai.chat.completions.create(
    model="llama3-8b-8192",  # Or another Groq-supported model
    messages=messages
    )

In [ ]:
def get_page_link(link):
    print(f"Getting page link for args: {link}", flush=True)  
    response = requests.get(link)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
    
        # Extract only readable text
        text = " ".join(soup.stripped_strings)
        return text[:3000]
    else:
        return f"Failed to retrieve page. Status code: {response.status_code}"  


In [ ]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        print(f"Arguments: {arguments}", flush=True)
        tool = globals().get(tool_name)
        print(f"Tool: {tool}", flush=True)
        
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [ ]:
get_page_link_json = {
    "name": "get_page_link",
    "description": "Always use this function when get the link asked first",
    "parameters": {
        "type": "object",
        "properties": {
            "link": {
                "type": "string",
                "description": "Link to the page",
            },
        },
        "required": ["link"],
        "additionalProperties": False
    }
}

In [ ]:
tools = [{"type": "function", "function": get_page_link_json}]
print(messages)

In [ ]:
def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = system_prompt + history + [{"role": "user", "content": message}]
    done = False
    while not done:

        # This is the call to the LLM - see that we pass in the tools json

        response = openai.chat.completions.create(model="llama3-8b-8192", messages=messages, tools=tools)

        finish_reason = response.choices[0].finish_reason
        print(f"Finish reason: {finish_reason}")
        # If the LLM wants to call a tool, we do that!
         
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            print(f"message: {message}", flush=True)
            
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            print(f"Tool results: {results}", flush=True)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

# Congratulations!

That was a small, simple step in the direction of Agentic AI, with your new environment!

Next time things get more interesting...